In [53]:
import twitter
import json

import py2neo
from py2neo import Graph, NodeMatcher
from py2neo.data import Node, Relationship

In [7]:
with open('config.json') as file:
    token = json.load(file)
token

{'CONSUMER_KEY': 'EChOOeWLzPLAjZ2MccF0yQCM9',
 'CONSUMER_SECRET': 'qZ8NfVO26bCZvEchmzkBZ2w9kHd1gPKQAICSihWefmiSxTSepB',
 'ACCESS_TOKEN': '976817437403357184-09Zxi32tC2oY04RDaVngtxh9Hcoc17x',
 'ACCESS_SECRET': 'qLvEDX7VBlXj9Y06l6oERlTHdRzvenQl56MR3vKbGIijH'}

In [8]:
api = twitter.Api(consumer_key=token['CONSUMER_KEY'],
                  consumer_secret=token['CONSUMER_SECRET'],
                  access_token_key=token['ACCESS_TOKEN'],
                  access_token_secret=token['ACCESS_SECRET'])

In [73]:
graph = Graph("bolt://localhost:11004", user="neo4j", password="123456")
graph.delete_all()

In [74]:
# api.GetUserTimeline(screen_name='nunenuh')
username = 'nunenuh'
user = api.GetUser(screen_name=username)
follower = api.GetFollowers(screen_name=username)
following = api.GetFriends(screen_name=username)

In [75]:
len(follower), len(following)

(161, 913)

In [51]:
user

User(ID=190368199, ScreenName=nunenuh)

In [76]:
#defining nodes and its relationship
nunenuh = Node("User", screen_name=user.screen_name, id=user.id)
tx = graph.begin()
tx.create(nunenuh)
tx.commit()

In [80]:
f = NodeMatcher(graph)
nunenuh = f.match("User", screen_name="nunenuh").first()

for fol in follower:
    fol_node = Node("User", screen_name=fol.screen_name, id=fol.id)
    fol_rel = Relationship(fol_node, "FOLLOWER", nunenuh)
    
    tx = graph.begin()
    tx.merge(fol_node, primary_label="User", primary_key="id")
    tx.create(fol_rel)
    tx.commit()
    

In [81]:
f = NodeMatcher(graph)
nunenuh = f.match("User", screen_name="nunenuh").first()

for foln in following:
    foln_node = Node("User", screen_name=foln.screen_name, id=foln.id)
    foln_rel = Relationship(nunenuh, "FOLLOWING", foln_node)
    
    tx = graph.begin()
    tx.merge(foln_node, primary_label="User", primary_key="id")
    tx.create(foln_rel)
    tx.commit()
    